<a href="https://colab.research.google.com/github/FlanagG21/DCC-Capstone/blob/main/Genius_API_Connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# prompt: mount to drive

from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/MyDrive/DCC Project/'


MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [58]:
#access token - need to remove before publishing
client_access_token = "OBFL9kbIUCq4GexThc8hO7cIjlvQ2BTFzd8rX_q1z8F1ZOtK1w8ExZKj1SobNAHL"

In [38]:
#generate url
search_term = "Missy Elliott"
genius_search_url = f"http://api.genius.com/search?q={search_term}&access_token={client_access_token}"
print(genius_search_url)

http://api.genius.com/search?q=Missy Elliott&access_token=oHyeePjpeKSzOGxJRFnHFHayfxzGi8CdcD4BvY1nrlQN_Ra1mE1KYGhEgHlcwibT


In [39]:
#get json data from response
import requests
response = requests.get(genius_search_url)
json_data = response.json()
#json_data

In [40]:
for song in json_data['response']['hits']:
    print(song['result']['full_title'])

Work It by Missy Elliott
WTF (Where They From) by Missy Elliott (Ft. Pharrell Williams)
Get Ur Freak On by Missy Elliott
Lose Control by Missy Elliott (Ft. Ciara & Fatman Scoop)
Gossip Folks by Missy Elliott (Ft. Ludacris)
The Rain (Supa Dupa Fly) by Missy Elliott
I'm Better by Missy Elliott (Ft. Cainon Lamb)
One Minute Man (Video Remix) by Missy Elliott (Ft. Ludacris & Trina)
Hot Boyz (Remix) by Missy Elliott (Ft. Eve, Lil' Mo, Nas & Q-Tip)
Bomb Intro / Pass That Dutch by Missy Elliott


In [41]:
#get top hit
top_result = json_data['response']['hits'][0]['result']
print(top_result['title'])

Work It


In [44]:
#FROM EMBEDDING FILE

import re
# to reduce vocab size let s try to remove more unecessary symbols and parts
# Define a cleaning function
def clean_lyrics(lyrics):
    # Remove section tags like [Intro], [Verse 1], etc.
    cleaned = re.sub(r'\[.*?\]', '', lyrics)

    # Remove credits or text after "---"
    cleaned = re.split(r'---', cleaned)[0]

    # Remove symbols except for line breaks (\n) and alphanumeric characters
    cleaned = re.sub(r'[^\w\s\n]', '', cleaned)

    # Remove extra whitespace and blank lines
    cleaned = re.sub(r'\n\s*\n', '\n', cleaned).strip()

    return cleaned

In [47]:
#get song ready for cleaning
print(top_result)
#song_df = [top_result['title'], top_result['tag'], top_result['artist_names']]
#print(song_df)

#df_clean['lyrics'] = df_clean['lyrics'].apply(clean_lyrics)
#print(df_clean['lyrics'])

{'annotation_count': 33, 'api_path': '/songs/4176', 'artist_names': 'Missy Elliott', 'full_title': 'Work It by\xa0Missy\xa0Elliott', 'header_image_thumbnail_url': 'https://images.genius.com/a54a36ef386a84ff0e689dc6d9cbdd18.300x300x1.jpg', 'header_image_url': 'https://images.genius.com/a54a36ef386a84ff0e689dc6d9cbdd18.953x953x1.jpg', 'id': 4176, 'lyrics_owner_id': 6654, 'lyrics_state': 'complete', 'path': '/Missy-elliott-work-it-lyrics', 'primary_artist_names': 'Missy Elliott', 'pyongs_count': 42, 'relationships_index_url': 'https://genius.com/Missy-elliott-work-it-sample', 'release_date_components': {'year': 2002, 'month': 9, 'day': 1}, 'release_date_for_display': 'September 1, 2002', 'release_date_with_abbreviated_month_for_display': 'Sep. 1, 2002', 'song_art_image_thumbnail_url': 'https://images.genius.com/a54a36ef386a84ff0e689dc6d9cbdd18.300x300x1.jpg', 'song_art_image_url': 'https://images.genius.com/a54a36ef386a84ff0e689dc6d9cbdd18.953x953x1.jpg', 'stats': {'unreviewed_annotations

In [49]:
print(json_data['response']['hits'][0])

{'highlights': [], 'index': 'song', 'type': 'song', 'result': {'annotation_count': 33, 'api_path': '/songs/4176', 'artist_names': 'Missy Elliott', 'full_title': 'Work It by\xa0Missy\xa0Elliott', 'header_image_thumbnail_url': 'https://images.genius.com/a54a36ef386a84ff0e689dc6d9cbdd18.300x300x1.jpg', 'header_image_url': 'https://images.genius.com/a54a36ef386a84ff0e689dc6d9cbdd18.953x953x1.jpg', 'id': 4176, 'lyrics_owner_id': 6654, 'lyrics_state': 'complete', 'path': '/Missy-elliott-work-it-lyrics', 'primary_artist_names': 'Missy Elliott', 'pyongs_count': 42, 'relationships_index_url': 'https://genius.com/Missy-elliott-work-it-sample', 'release_date_components': {'year': 2002, 'month': 9, 'day': 1}, 'release_date_for_display': 'September 1, 2002', 'release_date_with_abbreviated_month_for_display': 'Sep. 1, 2002', 'song_art_image_thumbnail_url': 'https://images.genius.com/a54a36ef386a84ff0e689dc6d9cbdd18.300x300x1.jpg', 'song_art_image_url': 'https://images.genius.com/a54a36ef386a84ff0e68

In [5]:
#DIRECT FROM KAGGLE DB
!pip install opendatasets
import opendatasets as od
import pandas as pd

In [7]:
od.download("https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information")

Dataset URL: https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information


In [9]:
songs = pd.read_csv("/content/genius-song-lyrics-with-language-information/song_lyrics.csv")
songs.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [19]:
#USER INPUT
title = "Souvenir"
artist = "Selena Gomez"

song = songs[(songs['title'] == title) & (songs['artist'] == artist)]
if not song.empty:
    print("Song found:")
    print(song)
else:
    print("Song not found.")

Song found:
            title  tag        artist  year   views features  \
3491193  Souvenir  pop  Selena Gomez  2020  272037       {}   

                                                    lyrics       id  \
3491193  [Intro]\nChills\n\n[Verse 1]\nNew York back in...  5279050   

        language_cld3 language_ft language  
3491193            en          en       en  


In [20]:
#clean the data
!git clone https://github.com/FlanagG21/DCC-Capstone.git
!pip install import_ipynb
import os
import import_ipynb

#os.chdir("/content/DCC-Capstone/")
#from songs_embedding import clean_lyrics

Cloning into 'DCC-Capstone'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 50 (delta 19), reused 18 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 457.52 KiB | 4.36 MiB/s, done.
Resolving deltas: 100% (19/19), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/reduce-lyrics/reduced_lyrics.csv'

In [23]:
import re
def clean_lyrics(lyrics):
    # Remove section tags like [Intro], [Verse 1], etc.
    cleaned = re.sub(r'\[.*?\]', '', lyrics)

    # Remove credits or text after "---"
    cleaned = re.split(r'---', cleaned)[0]

    # Remove symbols except for line breaks (\n) and alphanumeric characters
    cleaned = re.sub(r'[^\w\s\n]', '', cleaned)

    # Remove extra whitespace and blank lines
    cleaned = re.sub(r'\n\s*\n', '\n', cleaned).strip()

    return cleaned

In [26]:
# Apply the cleaning function
columns_to_drop = ['artist', 'views', 'features', 'language', 'language_cld3', 'language_ft', 'id','year']
song_clean = song.drop(columns=columns_to_drop)
song_clean.head()
song_clean['lyrics'] = song_clean['lyrics'].apply(clean_lyrics)
print(song_clean['lyrics'])

3491193    Chills\nNew York back in August tenth floor ba...
Name: lyrics, dtype: object


In [28]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Set up device for GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [31]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [29]:
# Load model from HuggingFace Hub and move to gpu
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = torch.nn.DataParallel(model)
model.to(device)
model.eval()  # Set model to evaluation mode

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

DataParallel(
  (module): MPNetModel(
    (embeddings): MPNetEmbeddings(
      (word_embeddings): Embedding(30527, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MPNetEncoder(
      (layer): ModuleList(
        (0-11): 12 x MPNetLayer(
          (attention): MPNetAttention(
            (attn): MPNetSelfAttention(
              (q): Linear(in_features=768, out_features=768, bias=True)
              (k): Linear(in_features=768, out_features=768, bias=True)
              (v): Linear(in_features=768, out_features=768, bias=True)
              (o): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
      

In [32]:

# Function to compute embedding
def compute_embedding(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)
    # Convert to list for Parquet compatibility
    return sentence_embedding.squeeze().numpy().tolist()

In [35]:
song['embedding'] = song['lyrics'].apply(compute_embedding)
print(song)

            title  tag        artist  year   views features  \
3491193  Souvenir  pop  Selena Gomez  2020  272037       {}   

                                                    lyrics       id  \
3491193  [Intro]\nChills\n\n[Verse 1]\nNew York back in...  5279050   

        language_cld3 language_ft language  \
3491193            en          en       en   

                                                 embedding  
3491193  [-0.010896549560129642, -0.004546075128018856,...  


<ipython-input-35-10317fdc00e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song['embedding'] = song['lyrics'].apply(compute_embedding)


In [38]:
#get movie recs
movies_df = pd.read_parquet(os.path.join(root, 'movies_embeddings.parquet'), engine='pyarrow')

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies(query_song_embedding, movies_df, top_k=5):
    # Convert movie embeddings to an array
    movie_embeddings = np.vstack(movies_df['embedding'].values)

    # Compute cosine similarity between the query song and all movie embeddings
    similarities = cosine_similarity([query_song_embedding], movie_embeddings)[0]

    # Get the indices of the top_k most similar movies
    top_indices = np.argsort(similarities)[::-1][:top_k]

    # Return the recommended movies with their similarity scores
    recommended_movies = movies_df.iloc[top_indices].copy()
    recommended_movies['similarity'] = similarities[top_indices]

    return recommended_movies

NameError: name 'root' is not defined

In [ ]:
import random


song_embedding = song['embedding']
recommendations = recommend_movies(song_embedding, movies_df)
recommended_movies_embeddings.append(recommendations['embedding'].values[0])
print(f"Recommendations for song '{random_song['title']}'':")
print(recommendations[['title', 'similarity']])